## Modèle avancé

In [1]:
from helpers import *

In [4]:
cols = ['sentiment', 'timestamp', 'date', "query", "username", "comment"]  
df = pd.read_csv("./../input/training.1600000.processed.noemoticon.csv", header=None, names=cols)

df = df.sample(n=100000, random_state=42)
len_df = len(df)
print(df.shape)
df.head()

(100000, 6)


,sentiment,timestamp,date,query,username,comment
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


In [5]:
df = clean_df(df)
print(df.shape)
df.head()

Processing: 100%|██████████| 100000/100000 [00:00<00:00, 730641.15it/s]


(92469, 5)


,comment,sentiment,comment_clean,words_nb,words_nb_clean
541200,@chrishasboobs AHHH I HOPE YOUR OK!!!,0,ahhh i hope your ok,6,5
750,"@misstoriblack cool , i have no tweet apps fo...",0,cool i have no tweet apps for my razr,12,9
766711,@TiannaChaos i know just family drama. its la...,0,i know just family drama its lamehey next time...,27,26
285055,School email won't open and I have geography ...,0,school email wont open and i have geography st...,16,15
379611,Going to miss Pastor's sermon on Faith...,0,going to miss pastors sermon on faith,7,7


### Word2Vec

In [6]:
df_w2vec = df.copy()

config = {
    "vector_size": 100,
    # Set input length to be the max from the number of words
    "input_length": df_w2vec["words_nb_clean"].max(),
    # Word before & after for word2vec
    "window": 5,
    # Max number of workers
    "workers": 8,
    # Vocal length
    "vocab_length": 70000
}

(model_w2vec, tokenizer_w2vec, callbacks_w2vec, X_w2vec, y_w2vec) = get_w2vec_data(config, df_w2vec)


W2vec Vocabulary Length (you can adjust the vocal length): 8196
Embedding Matrix Shape: (70000, 100)


2024-02-10 14:19:14.879017: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-10 14:19:14.879044: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2024-02-10 14:19:14.879053: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2024-02-10 14:19:14.879340: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-10 14:19:14.879749: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
model_w2vec.summary()

Model: "Sentiment_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 100)           7000000   
                                                                 
 bidirectional (Bidirection  (None, 40, 200)           160800    
 al)                                                             
                                                                 
 conv1d (Conv1D)             (None, 36, 100)           100100    
                                                                 
 global_max_pooling1d (Glob  (None, 100)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 16)                1616      
                                                                 
 dense_1 (Dense)             (None, 1)             

In [7]:
space = {
    "input_length": config["input_length"],
    'experiment_name': f"Tweet Sentiment - {len_df} - w2vec",
    'batch_size': hp.choice('batch_size', [128, 256]),  
    'epochs': hp.choice('epochs', [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),  
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.001),  
}

In [8]:
trials = Trials()
fmin(  
    fn=lambda params: objective(params, X_w2vec, y_w2vec, model_w2vec, callbacks_w2vec, tokenizer_w2vec),  
    space=space,  
    algo=tpe.suggest,  
    max_evals=1,
    trials=trials  
)


                                                  Running one fit with the params: 

                                                  {'batch_size': 256, 'epochs': 11, 'experiment_name': 'Tweet Sentiment - 100000 - w2vec', 'input_length': 40, 'learning_rate': 0.00030959048400700244}


2024/02/10 14:19:19 INFO mlflow.tracking.fluent: Experiment with name 'Tweet Sentiment - 100000 - w2vec' does not exist. Creating a new experiment.

Processing:   8%|▊         | 122896/1600000 [00:37<01:13, 20079.21it/s]


                                                  Epoch 1/11



2024-02-10 14:19:20.938953: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.



  1/228 [..............................] - ETA: 4:57 - loss: 0.6954 - accuracy: 0.4883

  2/228 [..............................] - ETA: 20s - loss: 0.6916 - accuracy: 0.5215 

  3/228 [..............................] - ETA: 20s - loss: 0.6896 - accuracy: 0.5195

  4/228 [..............................] - ETA: 20s - loss: 0.6903 - accuracy: 0.5146

  5/228 [..............................] - ETA: 20s - loss: 0.6863 - accuracy: 0.5297

  6/228 [..............................] - ETA: 20s - loss: 0.6843 - accuracy: 0.5404

  7/228 [..............................] - ETA: 20s - loss: 0.6820 - accuracy: 0.5519

  8/228 [>.............................] - ETA: 20s - loss: 0.6813 - accuracy: 0.5601

  9/228 [>.............................] - ETA: 20s - loss: 0.6783 - accuracy: 0.5720

 10/228 [>.............................] - ETA: 20s - loss: 0.6755 - accuracy: 0.5828

 11/228 [>.............................] - ETA: 20s - loss: 0.6743 - accuracy: 0.5856

 12/228 [>.............................]

{'batch_size': 1, 'epochs': 9, 'learning_rate': 0.00030959048400700244}

In [ ]:

with open("./model.pkl", "rb") as file:  
    model = pickle.load(file)
    
with open("./tokenizer.pkl", "rb") as file:  
    vector_model = pickle.load(file)

params = json.load(open("./params.json", 'r', encoding='utf-8'))
def pad_sequences(sequence, maxlen, value=0):  
    return np.array([np.pad(s[:maxlen], (max(0, maxlen-len(s)), 0), 'constant', constant_values=value) if len(s) < maxlen else s[:maxlen] for s in sequence])  
  
def predict(comment, model, vector_model):
    # We need to pad sequences to ensure uniform input size  
    sequence = tokenizer_w2vec.texts_to_sequences(comment)
    padded_sequence = pad_sequences(sequence, maxlen=int(params["input_length"]))

    prediction = model.predict(padded_sequence)
    # print(prediction)
    sentiment = prediction.astype(float)[0][0]
    return sentiment.item()
    
def comment_to_vec(comment, model):
    vec = np.zeros(100)
    num_words = 0
    for word in comment:
        if word in model.wv:
            vec += model.wv[word]
            num_words += 1
    if num_words > 0:
        vec /= num_words
    return vec

with tf.device('/CPU:0'):
    print(predict(["I am so sad, this is very bad news, terrible!"], model, tokenizer_w2vec))
    print(predict(["I am so happy, this is very good news, congrats!"], model, tokenizer_w2vec))


In [ ]:
# We need to pad sequences to ensure uniform input size  
sequence = tokenizer_w2vec.texts_to_sequences(df["comment_clean"])
padded_sequence = pad_sequences(sequence, maxlen=int(params["input_length"]))

prediction = model.predict(padded_sequence)

In [ ]:
y_pred_rounded = (prediction > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)
accuracy = accuracy_score(df["sentiment"], y_pred_rounded)

### Glove

In [ ]:
df_glove = df.copy()

config = {
    # Best to use 100 since I'm using glove 100 of pretrained data
    "vector_size": 100,
    # Set input length to be the max from the number of words
    "input_length": df_glove["words_nb_clean"].max(),
    # Word before & after for word2vec
    "window": 5,
    # Max number of workers
    "workers": 8,
    # Vocal length
    "vocab_length": 70000
}

(model_glove, tokenizer_glove, callbacks_glove, X_glove, y_glove) = get_glove_data(config, df_glove)


In [ ]:
model_glove.summary()

In [ ]:
space = {
    "input_length": config["input_length"],
    'experiment_name': f"Tweet Sentiment - {len_df} - glove",
    'batch_size': hp.choice('batch_size', [128, 256]),  
    'epochs': hp.choice('epochs', [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),  
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.001),  
}

In [ ]:
trials = Trials()
fmin(  
    fn=lambda params: objective(params, X_glove, y_glove, model_glove, callbacks_glove, tokenizer_glove),  
    space=space,  
    algo=tpe.suggest,
    max_evals=10,
    trials=trials  
)